In [ ]:
### Color Map
# colors_map = np.array(
#     [
#         [0,   0,   0, 255],  # 0 undefined
#         [255, 158, 0, 255],  # 1 car  orange
#         [0, 0, 230, 255],    # 2 pedestrian  Blue
#         [47, 79, 79, 255],   # 3 sign  Darkslategrey
#         [220, 20, 60, 255],  # 4 CYCLIST  Crimson
#         [255, 69, 0, 255],   # 5 traiffic_light  Orangered
#         [255, 140, 0, 255],  # 6 pole  Darkorange
#         [233, 150, 70, 255], # 7 construction_cone  Darksalmon
#         [255, 61, 99, 255],  # 8 bycycle  Red
#         [112, 128, 144, 255],# 9 motorcycle  Slategrey
#         [222, 184, 135, 255],# 10 building Burlywood
#         [0, 175, 0, 255],    # 11 vegetation  Green
#         [165, 42, 42, 255],  # 12 trunk  nuTonomy green
#         [0, 207, 191, 255],  # 13 curb, road, lane_marker, other_ground
#         [75, 0, 75, 255], # 14 walkable, sidewalk
#         [255, 0, 0, 255], # 15 unobsrvd
#         [0, 0, 0, 0],  # 16 undefined
#         [0, 0, 0, 0],  # 16 undefined
#     ])

In [ ]:
## Test Dataloader: Load data
from ssc_pl.data.datasets.semantic_kitti import *
from ssc_pl.data.datasets.occ3d import *
from torch.utils.data import Dataset, DataLoader

kittidata = './data/kitti/dataset'
kittilabel = './data/kitti/dataset/dataset/labels'
kittidepth = './data/kitti/dataset/dataset/depth'
dataroot = './data/nuscenes'

dataset = Occ3D(split = 'train', data_root=dataroot)
kitti = SemanticKITTI(split='val', data_root=kittidata, label_root=kittilabel, depth_root=kittidepth)
#dataset = DataLoader(data, batch_size=16, shuffle=True)

data, label = dataset[0]
ex_data = data['cam_channels']


In [ ]:
# # SemanticKITTI example
# from ssc_pl.models.utils import (cumprod, flatten_fov_from_voxels, flatten_multi_scale_feats, generate_grid,
#                      get_level_start_index, index_fov_back_to_voxels, interpolate_flatten,
#                      nchw_to_nlc, nlc_to_nchw, pix2vox)
# image_shape = (370, 1220)
# image_grid = generate_grid(image_shape)
# image_grid = torch.flip(image_grid, dims=[0]).unsqueeze(0)
# depth = kitdat['depth'].unsqueeze(0)
# ### bs x 900 x 1600

# K = kitdat['cam_K'].unsqueeze(0)
# ### bs x 3 x 3

# E = kitdat['cam_pose'].unsqueeze(0)
# voxel_origin = kitdat['voxel_origin'].unsqueeze(0)
# vol_pts_kit = pix2vox(
#     image_grid,
#     depth.unsqueeze(0),
#     K,
#     E,
#     voxel_origin,
#     0.2,
#     downsample_z=2).long()

# scene_size = (256, 256, 32) ## in voxel pixels
# points = vol_pts_kit
# keep = ((points[..., 0] >= 0) & (points[..., 0] < scene_size[0]) &
#         (points[..., 1] >= 0) & (points[..., 1] < scene_size[1]) &
#         (points[..., 2] >= 0) & (points[..., 2] < scene_size[2]))
# assert points.shape[0] == 1
# geom = points.squeeze()[keep.squeeze()]
# kitti_pts_mask = torch.zeros(scene_size, dtype=torch.bool)
# kitti_pts_mask[geom[:, 0], geom[:, 1], geom[:, 2]] = True

In [ ]:
# ## KITTI- Visualize depth to voxel
# from pythreejs import *
# import numpy as np
# from IPython.display import display

# # pts_mask
# h, w, z = torch.nonzero(kitti_pts_mask, as_tuple=True)  
# kitti_points = torch.stack((h, w, z), dim=-1).numpy()

# pcd_geometry = BufferGeometry(
#     attributes={
#         'position': BufferAttribute(kitti_points, normalized=False),
#         #'color': BufferAttribute(colors, normalized=False),
#     }
# )

# material = PointsMaterial(color='blue', size=0.5)
# scene_data = Points(geometry=pcd_geometry, material=material)
# # scene = Scene(children=[
# #     scene_data,
# #     AmbientLight(color='#cccccc'),
# #     DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
# # ])
# scene = Scene(children=[
#     scene_data,
#     AmbientLight(color='#cccccc'),
#     DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
# ])

# camera = PerspectiveCamera(position=[0, 100, 0], fov=75, near=0.1)
# camera.up = [0, 1, 0]

# controller = OrbitControls(controlling=camera)

# renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=800, height=800)

# display(renderer)

In [ ]:
## Plot Sample Image from data

import matplotlib.pyplot as plt

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

img = np.array(ex_data['CAM_FRONT']['img'], dtype=np.float32)

img = img * std[:, None, None] + mean[:, None, None]
img = (img * 255).astype(np.uint8)
img = img.transpose(1, 2, 0)

plt.imshow(img)
plt.axis('off')

In [ ]:
## Testing voxel proposal - get voxel from depth data

from ssc_pl.models.utils import (cumprod, flatten_fov_from_voxels, flatten_multi_scale_feats, generate_grid,
                     get_level_start_index, index_fov_back_to_voxels, interpolate_flatten,
                     nchw_to_nlc, nlc_to_nchw, pix2vox)

image_shape = (900, 1600)
image_grid = generate_grid(image_shape)
image_grid = torch.flip(image_grid, dims=[0]).unsqueeze(0)
#image_grid = torch.flip(image_grid, dims=[3])
### bs x 2 x 900 x 1600
voxel_origin = np.array((-40, -40, -2)) # in meters (Width Depth Height)
voxel_origin = torch.from_numpy(voxel_origin).unsqueeze(0)
### bs x 3

voxel_size = 0.4
vol_pts = []

for cam in ex_data:
    depth = ex_data[cam]['depth'].unsqueeze(0)
    #depth = torch.flip(depth, dims = [1])
    ### bs x 900 x 1600

    K = ex_data[cam]['cam_K'].unsqueeze(0)
    ### bs x 3 x 3

    E = ex_data[cam]['cam_pose'].unsqueeze(0)
    ### bs x 4 x 4

    vol_pts_cam = pix2vox(
        image_grid,
        depth.unsqueeze(0),
        K,
        E,
        voxel_origin,
        voxel_size,
        downsample_z=1).long()

    min_h = torch.min(vol_pts_cam[:, :, 2])
    vol_pts_cam[:, :, 2] -= min_h
    vol_pts.append(vol_pts_cam)
    ### bs x HW x 3

points = torch.cat(vol_pts, dim=1)


scene_size = (200, 200, 16) ## in voxel pixels
keep = ((points[..., 0] >= 0) & (points[..., 0] < scene_size[0]) &
        (points[..., 1] >= 0) & (points[..., 1] < scene_size[1]) &
        (points[..., 2] >= 0) & (points[..., 2] < scene_size[2]))
assert points.shape[0] == 1
geom = points.squeeze()[keep.squeeze()]
pts_mask = torch.zeros(scene_size, dtype=torch.bool)
pts_mask[geom[:, 0], geom[:, 1], geom[:, 2]] = True


In [ ]:
## Visualize depth to voxel
from pythreejs import *
import numpy as np
from IPython.display import display

# pts_mask
h, w, z = torch.nonzero(pts_mask, as_tuple=True)  
points = torch.stack((h, w, z), dim=-1).numpy()
print(points.shape)
pcd_geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(points, normalized=False),
        #'color': BufferAttribute(colors, normalized=False),
    }
)

material = PointsMaterial(color='blue', size=0.5)
scene_data = Points(geometry=pcd_geometry, material=material)
# scene = Scene(children=[
#     scene_data,
#     AmbientLight(color='#cccccc'),
#     DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
# ])
scene = Scene(children=[
    scene_data,
    AmbientLight(color='#cccccc'),
    DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
])

camera = PerspectiveCamera(position=[0, 0, 100], fov=75, near=0.1)
camera.up = [0, 0, 1]

controller = OrbitControls(controlling=camera)

renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=500, height=500)

display(renderer)


In [ ]:
# Render GT label
gt = label['target']

gt = gt * np.array(pts_mask)
boolean_tensor = gt != 0

x, y, z = np.nonzero(boolean_tensor)  
points = np.stack((x, y, z)).T

geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(points, normalized=False),
    }
)

material = PointsMaterial(vertexColors='VertexColors', size=1)
point_cloud = Points(geometry=geometry, material=material)

scene = Scene(children=[
    point_cloud,
    AmbientLight(color='#cccccc'),
    DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
])

# camera = PerspectiveCamera(position=[-100, 0, 0], fov=75, near=0.1, far=1000)
# camera.up = [0, 0, 1]

# controller = OrbitControls(controlling=camera)

renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=500, height=500)

renderer

In [ ]:
# # Render Raw depth points
# vol_pts = []
# for i in range(6):
#     points = np.load(f'points_{i}.npy')
#     vol_pts.append(points)


# colors = np.zeros_like(points)

# scene_data = []
# for i in range(len(vol_pts)):
#     points = vol_pts[i].squeeze()

#     colors = np.zeros_like(points)

#     pcd_geometry = BufferGeometry(
#         attributes={
#             'position': BufferAttribute(points, normalized=False),
#             'color': BufferAttribute(colors, normalized=False),
#         }
#     )


#     material = PointsMaterial(vertexColors='VertexColors', size=0.5)
#     scene_data.append(Points(geometry=pcd_geometry, material=material))
# scene = Scene(children=[
#     *scene_data,
#     AmbientLight(color='#cccccc'),
#     DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)
# ])

# camera = PerspectiveCamera(position=[0, 0, 100], fov=75)
# camera.up = [0, -1, 0]

# controller = OrbitControls(controlling=camera)

# renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=1000, height=800)

# renderer